In [1]:
from functions import *
import time

# Data

In [2]:
dataset = "ProximalPhalanxOutlineAgeGroup"

In [3]:
path       = '/home/keunsu/UCRArchive_2018' + "/" + dataset + "/"

In [4]:
X1_sample = pd.read_csv(path + dataset + "_TRAIN.tsv", sep="\t", header=None)
X2_sample = pd.read_csv(path + dataset + "_TEST.tsv",  sep="\t", header=None)


X1 = np.array(X1_sample)
X2 = np.array(X2_sample)
X = np.vstack([X1, X2])
Ntrain = len(X1)

label, data = X[:,0], X[:,1:]

In [5]:
dims = [2, 3, 4, 5, 10, 20]
taus = [1, 2, 3, 4, 5, 10, 20]
resolutions = [10, 50] 
max_frequencies = [10,20,40]    
bandwidths = [0.1,1]
seeds = None

# Fourier

In [13]:
all_results = []

for max_frequency in max_frequencies:

    result_Fourier = np.zeros((len(data), max_frequency), dtype=float)
    for i in range(len(data)):
        result_Fourier[i] = (np.abs(np.fft.fft(data[i])) / data.shape[1])[1:max_frequency+1]


   
    acc_rate = []

    if seeds == None:
        acc_rate.append(train_eval_linear_svm(result_Fourier, label, Ntrain, gamma_auto=True))
        
    else:
        for seed in range(len(seeds)):
            result_Fourier_shuffled, label_shuffled = shuffle(result_Fourier, label, random_state=seed)
    
            acc_rate.append(train_eval_linear_svm(result_Fourier_shuffled, label_shuffled, Ntrain, gamma_auto=True))
    

    mean_acc = float(np.mean(acc_rate))
    std_acc  = float(np.std(acc_rate, ddof=1)) if len(acc_rate) > 1 else 0.0

    result = {
        "max_frequency": max_frequency,
        "mean_acc": mean_acc,
        "std_acc": std_acc,
        "acc_rate": acc_rate,
    }
    all_results.append(result)

    print(f"[max_freq={max_frequency:>3}] mean={mean_acc:.2f}% (std={std_acc:.2f}%)")


print("\n=== Summary over max_frequency (sorted by mean accuracy) ===")
all_results_sorted = sorted(all_results, key=lambda r: r["mean_acc"], reverse=True)
for r in all_results_sorted:
    print(f"max_freq={r['max_frequency']:>3} | mean={r['mean_acc']:.2f}% (std={r['std_acc']:.2f}%)")

[max_freq= 10] mean=82.93% (std=0.00%)
[max_freq= 20] mean=84.39% (std=0.00%)
[max_freq= 40] mean=82.93% (std=0.00%)

=== Summary over max_frequency (sorted by mean accuracy) ===
max_freq= 20 | mean=84.39% (std=0.00%)
max_freq= 10 | mean=82.93% (std=0.00%)
max_freq= 40 | mean=82.93% (std=0.00%)


# Method A

In [14]:
method = "betti"
norm = 'chebyshev'
 
all_results = []

for resolution in resolutions:  
    print(f"\n### Running experiments with resolution = {resolution} ###")
    for dim in dims:
        for tau in taus:
            if dim * tau < data.shape[1]:


                accs = SW_method(
                    data, label, Ntrain=Ntrain,
                    resolution=resolution, dim=dim, tau=tau,
                    method=method, norm=norm,
                    seeds=seeds,
                    n_jobs_seeds=-1      
                )


            
                mean_acc = float(np.mean(accs))
                std_acc = float(np.std(accs, ddof=1)) if len(accs) > 1 else 0.0
    
                result = {
                    "resolution": resolution,  
                    "dim": dim,
                    "tau": tau,
                    "mean_acc": mean_acc,
                    "std_acc": std_acc,
                    "acc_list": accs,
                }
                all_results.append(result)
    
                print(f"[res={resolution:>3}, dim={dim:>3}, tau={tau}]"
                      f"mean={mean_acc:.2f}% (std={std_acc:.2f}%)")

            else:
                continue


print("\n=== Summary (sorted by mean accuracy) ===")
all_results_sorted = sorted(all_results, key=lambda r: r["mean_acc"], reverse=True)
for r in all_results_sorted:
    print(f"res={r['resolution']:>3}, dim={r['dim']:>3}, tau={r['tau']} | "
          f"mean={r['mean_acc']:.2f}% (std={r['std_acc']:.2f}%)")



notebook_name = ipynbname.name()  

filename = f"{notebook_name}_results_{method}_{norm}.pkl"

with open(filename, "wb") as f:
    pickle.dump(all_results, f)


### Running experiments with resolution = 10 ###
[res= 10, dim=  2, tau=1]mean=83.41% (std=0.00%)
[res= 10, dim=  2, tau=2]mean=81.95% (std=0.00%)
[res= 10, dim=  2, tau=3]mean=81.46% (std=0.00%)
[res= 10, dim=  2, tau=4]mean=82.44% (std=0.00%)
[res= 10, dim=  2, tau=5]mean=80.00% (std=0.00%)
[res= 10, dim=  2, tau=10]mean=81.95% (std=0.00%)
[res= 10, dim=  2, tau=20]mean=79.02% (std=0.00%)
[res= 10, dim=  3, tau=1]mean=81.95% (std=0.00%)
[res= 10, dim=  3, tau=2]mean=81.95% (std=0.00%)
[res= 10, dim=  3, tau=3]mean=83.90% (std=0.00%)
[res= 10, dim=  3, tau=4]mean=82.44% (std=0.00%)
[res= 10, dim=  3, tau=5]mean=81.46% (std=0.00%)
[res= 10, dim=  3, tau=10]mean=80.49% (std=0.00%)
[res= 10, dim=  3, tau=20]mean=74.63% (std=0.00%)
[res= 10, dim=  4, tau=1]mean=81.95% (std=0.00%)
[res= 10, dim=  4, tau=2]mean=83.90% (std=0.00%)
[res= 10, dim=  4, tau=3]mean=83.90% (std=0.00%)
[res= 10, dim=  4, tau=4]mean=84.39% (std=0.00%)
[res= 10, dim=  4, tau=5]mean=82.93% (std=0.00%)
[res= 10, dim= 

NameError: name 'ipynbname' is not defined

In [17]:
times = []

for i in range(10): 
    start = time.perf_counter()

    tde = TimeDelayEmbedding(dim=5, delay=2, skip=1)
    emb_all = np.stack([tde(ts) for ts in data], axis=0)

    VR = VietorisRipsPersistence(
        metric='chebyshev',
        coeff=11,
        homology_dimensions=[1],
        n_jobs=-1
    )
    PDs_raw = VR.fit_transform(emb_all)
    PDs = [D[:, :2] if (D is not None and D.size) else np.empty((0, 2), dtype=dtype)
           for D in PDs_raw]

    elapsed = time.perf_counter() - start
    times.append(elapsed)
    print(f"Run {i+1:02d}: {elapsed:.2f} sec")

print("\n=== Summary ===")
print(f"Mean: {np.mean(times):.2f} sec")
print(f"Std:  {np.std(times):.2f} sec")

Run 01: 0.14 sec
Run 02: 0.14 sec
Run 03: 0.14 sec
Run 04: 0.14 sec
Run 05: 0.14 sec
Run 06: 0.14 sec
Run 07: 0.14 sec
Run 08: 0.14 sec
Run 09: 0.14 sec
Run 10: 0.15 sec

=== Summary ===
Mean: 0.14 sec
Std:  0.00 sec


# Method B

In [6]:
method = "image"
norm = 'chebyshev'

all_results = []

for resolution in resolutions: 
    print(f"\n### Running experiments with resolution = {resolution} ###")
    for dim in dims:
        for tau in taus:
            if dim * tau < data.shape[1]:
                for bw in bandwidths:  
                    accs = SW_method(
                        data, label, Ntrain=Ntrain,
                        resolution=resolution, dim=dim, tau=tau,
                        method=method, norm=norm, bandwidth=bw,
                        seeds=seeds,
                        n_jobs_seeds=-1      
                    )

                    mean_acc = float(np.mean(accs))
                    std_acc = float(np.std(accs, ddof=1)) if len(accs) > 1 else 0.0

                    result = {
                        "resolution": resolution,
                        "dim": dim,
                        "tau": tau,
                        "bandwidth": bw,         
                        "mean_acc": mean_acc,
                        "std_acc": std_acc,
                        "acc_list": accs,
                    }
                    all_results.append(result)

                    print(
                        f"[res={resolution:>3}, dim={dim:>3}, tau={tau}, bw={bw}] "
                        f"mean={mean_acc:.2f}% (std={std_acc:.2f}%)"
                    )
            else:
                continue


print("\n=== Summary (sorted by mean accuracy) ===")
all_results_sorted = sorted(all_results, key=lambda r: r["mean_acc"], reverse=True)
for r in all_results_sorted:
    print(
        f"res={r['resolution']:>3}, dim={r['dim']:>3}, tau={r['tau']}, "
        f"bw={r['bandwidth']} | "
        f"mean={r['mean_acc']:.2f}% (std={r['std_acc']:.2f}%)"
    )


notebook_name = ipynbname.name()  

filename = f"{notebook_name}_results_{method}_{norm}.pkl"

with open(filename, "wb") as f:
    pickle.dump(all_results, f)


### Running experiments with resolution = 10 ###
[res= 10, dim=  2, tau=1, bw=0.1] mean=82.44% (std=0.00%)
[res= 10, dim=  2, tau=1, bw=1] mean=57.56% (std=0.00%)
[res= 10, dim=  2, tau=2, bw=0.1] mean=84.88% (std=0.00%)
[res= 10, dim=  2, tau=2, bw=1] mean=73.66% (std=0.00%)
[res= 10, dim=  2, tau=3, bw=0.1] mean=83.41% (std=0.00%)
[res= 10, dim=  2, tau=3, bw=1] mean=83.90% (std=0.00%)
[res= 10, dim=  2, tau=4, bw=0.1] mean=81.46% (std=0.00%)
[res= 10, dim=  2, tau=4, bw=1] mean=83.90% (std=0.00%)
[res= 10, dim=  2, tau=5, bw=0.1] mean=82.44% (std=0.00%)
[res= 10, dim=  2, tau=5, bw=1] mean=83.90% (std=0.00%)
[res= 10, dim=  2, tau=10, bw=0.1] mean=82.93% (std=0.00%)
[res= 10, dim=  2, tau=10, bw=1] mean=83.41% (std=0.00%)
[res= 10, dim=  2, tau=20, bw=0.1] mean=81.46% (std=0.00%)
[res= 10, dim=  2, tau=20, bw=1] mean=84.88% (std=0.00%)
[res= 10, dim=  3, tau=1, bw=0.1] mean=84.39% (std=0.00%)
[res= 10, dim=  3, tau=1, bw=1] mean=71.71% (std=0.00%)
[res= 10, dim=  3, tau=2, bw=0.1] 

NameError: name 'ipynbname' is not defined

In [19]:
times = []

for i in range(10): 
    start = time.perf_counter()

    tde = TimeDelayEmbedding(dim=5, delay=2, skip=1)
    emb_all = np.stack([tde(ts) for ts in data], axis=0)

    VR = VietorisRipsPersistence(
        metric='chebyshev',
        coeff=11,
        homology_dimensions=[1],
        n_jobs=-1
    )
    PDs_raw = VR.fit_transform(emb_all)
    PDs = [D[:, :2] if (D is not None and D.size) else np.empty((0, 2), dtype=dtype)
           for D in PDs_raw]

    elapsed = time.perf_counter() - start
    times.append(elapsed)
    print(f"Run {i+1:02d}: {elapsed:.2f} sec")

print("\n=== Summary ===")
print(f"Mean: {np.mean(times):.2f} sec")
print(f"Std:  {np.std(times):.2f} sec")

Run 01: 0.14 sec
Run 02: 0.14 sec
Run 03: 0.14 sec
Run 04: 0.14 sec
Run 05: 0.14 sec
Run 06: 0.15 sec
Run 07: 0.14 sec
Run 08: 0.14 sec
Run 09: 0.14 sec
Run 10: 0.14 sec

=== Summary ===
Mean: 0.14 sec
Std:  0.00 sec


# Method C

In [20]:
method = "betti"
norm = 'euclidean'

all_results = []

for resolution in resolutions:  
    print(f"\n### Running experiments with resolution = {resolution} ###")
    for dim in dims:
        for tau in taus:
            if dim * tau < data.shape[1]:


                accs = SW_method(
                    data, label, Ntrain=Ntrain,
                    resolution=resolution, dim=dim, tau=tau,
                    method=method, norm=norm,
                    seeds=seeds,
                    n_jobs_seeds=-1      
                )


            
                mean_acc = float(np.mean(accs))
                std_acc = float(np.std(accs, ddof=1)) if len(accs) > 1 else 0.0
    
                result = {
                    "resolution": resolution,  
                    "dim": dim,
                    "tau": tau,
                    "mean_acc": mean_acc,
                    "std_acc": std_acc,
                    "acc_list": accs,
                }
                all_results.append(result)
    
                print(f"[res={resolution:>3}, dim={dim:>3}, tau={tau}]"
                      f"mean={mean_acc:.2f}% (std={std_acc:.2f}%)")

            else:
                continue


print("\n=== Summary (sorted by mean accuracy) ===")
all_results_sorted = sorted(all_results, key=lambda r: r["mean_acc"], reverse=True)
for r in all_results_sorted:
    print(f"res={r['resolution']:>3}, dim={r['dim']:>3}, tau={r['tau']} | "
          f"mean={r['mean_acc']:.2f}% (std={r['std_acc']:.2f}%)")



notebook_name = ipynbname.name()  

filename = f"{notebook_name}_results_{method}_{norm}.pkl"

with open(filename, "wb") as f:
    pickle.dump(all_results, f)


### Running experiments with resolution = 10 ###
[res= 10, dim=  2, tau=1]mean=81.95% (std=0.00%)
[res= 10, dim=  2, tau=2]mean=78.54% (std=0.00%)
[res= 10, dim=  2, tau=3]mean=82.44% (std=0.00%)
[res= 10, dim=  2, tau=4]mean=81.46% (std=0.00%)
[res= 10, dim=  2, tau=5]mean=83.90% (std=0.00%)
[res= 10, dim=  2, tau=10]mean=83.90% (std=0.00%)
[res= 10, dim=  2, tau=20]mean=78.54% (std=0.00%)
[res= 10, dim=  3, tau=1]mean=79.51% (std=0.00%)
[res= 10, dim=  3, tau=2]mean=80.00% (std=0.00%)
[res= 10, dim=  3, tau=3]mean=82.93% (std=0.00%)
[res= 10, dim=  3, tau=4]mean=81.46% (std=0.00%)
[res= 10, dim=  3, tau=5]mean=82.44% (std=0.00%)
[res= 10, dim=  3, tau=10]mean=77.56% (std=0.00%)
[res= 10, dim=  3, tau=20]mean=80.49% (std=0.00%)
[res= 10, dim=  4, tau=1]mean=82.93% (std=0.00%)
[res= 10, dim=  4, tau=2]mean=82.93% (std=0.00%)
[res= 10, dim=  4, tau=3]mean=80.00% (std=0.00%)
[res= 10, dim=  4, tau=4]mean=83.41% (std=0.00%)
[res= 10, dim=  4, tau=5]mean=84.39% (std=0.00%)
[res= 10, dim= 

NameError: name 'ipynbname' is not defined

In [22]:
times = []

for i in range(10):  
    start = time.perf_counter()

    tde = TimeDelayEmbedding(dim=5, delay=5, skip=1)
    emb_all = np.stack([tde(ts) for ts in data], axis=0)

    VR = VietorisRipsPersistence(
        metric='euclidean',
        coeff=11,
        homology_dimensions=[1],
        n_jobs=-1
    )
    PDs_raw = VR.fit_transform(emb_all)
    PDs = [D[:, :2] if (D is not None and D.size) else np.empty((0, 2), dtype=dtype)
           for D in PDs_raw]

    elapsed = time.perf_counter() - start
    times.append(elapsed)
    print(f"Run {i+1:02d}: {elapsed:.2f} sec")

print("\n=== Summary ===")
print(f"Mean: {np.mean(times):.2f} sec")
print(f"Std:  {np.std(times):.2f} sec")

Run 01: 0.12 sec
Run 02: 0.12 sec
Run 03: 0.12 sec
Run 04: 0.12 sec
Run 05: 0.12 sec
Run 06: 0.12 sec
Run 07: 0.12 sec
Run 08: 0.12 sec
Run 09: 0.12 sec
Run 10: 0.13 sec

=== Summary ===
Mean: 0.12 sec
Std:  0.00 sec


# Method D

In [23]:
method = "image"
norm = 'euclidean'

all_results = []

for resolution in resolutions: 
    print(f"\n### Running experiments with resolution = {resolution} ###")
    for dim in dims:
        for tau in taus:
            if dim * tau < data.shape[1]:
                for bw in bandwidths:  
                    accs = SW_method(
                        data, label, Ntrain=Ntrain,
                        resolution=resolution, dim=dim, tau=tau,
                        method=method, norm=norm, bandwidth=bw,
                        seeds=seeds,
                        n_jobs_seeds=-1      
                    )

                    mean_acc = float(np.mean(accs))
                    std_acc = float(np.std(accs, ddof=1)) if len(accs) > 1 else 0.0

                    result = {
                        "resolution": resolution,
                        "dim": dim,
                        "tau": tau,
                        "bandwidth": bw,         
                        "mean_acc": mean_acc,
                        "std_acc": std_acc,
                        "acc_list": accs,
                    }
                    all_results.append(result)

                    print(
                        f"[res={resolution:>3}, dim={dim:>3}, tau={tau}, bw={bw}] "
                        f"mean={mean_acc:.2f}% (std={std_acc:.2f}%)"
                    )
            else:
                continue


print("\n=== Summary (sorted by mean accuracy) ===")
all_results_sorted = sorted(all_results, key=lambda r: r["mean_acc"], reverse=True)
for r in all_results_sorted:
    print(
        f"res={r['resolution']:>3}, dim={r['dim']:>3}, tau={r['tau']}, "
        f"bw={r['bandwidth']} | "
        f"mean={r['mean_acc']:.2f}% (std={r['std_acc']:.2f}%)"
    )


notebook_name = ipynbname.name()  

filename = f"{notebook_name}_results_{method}_{norm}.pkl"

with open(filename, "wb") as f:
    pickle.dump(all_results, f)


### Running experiments with resolution = 10 ###
[res= 10, dim=  2, tau=1, bw=0.1] mean=84.39% (std=0.00%)
[res= 10, dim=  2, tau=1, bw=1] mean=59.02% (std=0.00%)
[res= 10, dim=  2, tau=2, bw=0.1] mean=81.46% (std=0.00%)
[res= 10, dim=  2, tau=2, bw=1] mean=83.41% (std=0.00%)
[res= 10, dim=  2, tau=3, bw=0.1] mean=80.49% (std=0.00%)
[res= 10, dim=  2, tau=3, bw=1] mean=81.95% (std=0.00%)
[res= 10, dim=  2, tau=4, bw=0.1] mean=77.07% (std=0.00%)
[res= 10, dim=  2, tau=4, bw=1] mean=81.46% (std=0.00%)
[res= 10, dim=  2, tau=5, bw=0.1] mean=82.93% (std=0.00%)
[res= 10, dim=  2, tau=5, bw=1] mean=85.85% (std=0.00%)
[res= 10, dim=  2, tau=10, bw=0.1] mean=80.49% (std=0.00%)
[res= 10, dim=  2, tau=10, bw=1] mean=80.98% (std=0.00%)
[res= 10, dim=  2, tau=20, bw=0.1] mean=81.95% (std=0.00%)
[res= 10, dim=  2, tau=20, bw=1] mean=80.49% (std=0.00%)
[res= 10, dim=  3, tau=1, bw=0.1] mean=82.44% (std=0.00%)
[res= 10, dim=  3, tau=1, bw=1] mean=78.05% (std=0.00%)
[res= 10, dim=  3, tau=2, bw=0.1] 

NameError: name 'ipynbname' is not defined

In [24]:
times = []

for i in range(10):  
    start = time.perf_counter()

    tde = TimeDelayEmbedding(dim=2, delay=5, skip=1)
    emb_all = np.stack([tde(ts) for ts in data], axis=0)

    VR = VietorisRipsPersistence(
        metric='euclidean',
        coeff=11,
        homology_dimensions=[1],
        n_jobs=-1
    )
    PDs_raw = VR.fit_transform(emb_all)
    PDs = [D[:, :2] if (D is not None and D.size) else np.empty((0, 2), dtype=dtype)
           for D in PDs_raw]

    elapsed = time.perf_counter() - start
    times.append(elapsed)
    print(f"Run {i+1:02d}: {elapsed:.2f} sec")

print("\n=== Summary ===")
print(f"Mean: {np.mean(times):.2f} sec")
print(f"Std:  {np.std(times):.2f} sec")

Run 01: 0.15 sec
Run 02: 0.15 sec
Run 03: 0.14 sec
Run 04: 0.14 sec
Run 05: 0.14 sec
Run 06: 0.14 sec
Run 07: 0.15 sec
Run 08: 0.14 sec
Run 09: 0.15 sec
Run 10: 0.15 sec

=== Summary ===
Mean: 0.15 sec
Std:  0.00 sec


# DTW

In [7]:
results, accs = run_knn_dtw_parallel(data, label, Ntrain, seeds=seeds, n_jobs=-1)
std = float(np.std(accs, ddof=1)) if len(accs) > 1 else 0.0

print("Accuracy for each seed (%) =", (accs*100).round(2).tolist())
print("Mean (%) =", accs.mean()*100, " | Std. dev. (%) =", std*100)

Accuracy for each seed (%) = [80.49]
Mean (%) = 80.48780487804879  | Std. dev. (%) = 0.0


# EMPH-Betti (SR)

In [27]:
trials = 100
n_jobs = -1                        
method = 'betti'            



all_results = [] 

for resolution in resolutions:
    for fmax in max_frequencies:
        
        
        fft_mag = np.abs(np.fft.fft(data, axis=1))[:, 1:fmax+1]  
        base_amp = (2 * np.sqrt(3) * fft_mag) / data.shape[1]               

        results = Parallel(n_jobs=n_jobs, backend="loky", verbose=0)(
            delayed(EMPH_method_original)(
                tr, resolution, fmax, base_amp, label=label, Ntrain=Ntrain,
                method=method,
               seeds = seeds
            )
            for tr in range(trials)
        )


        accs_rays = np.mean(np.array([r[0] for r in results]), axis=1)
        best_idx = int(np.argmax(accs_rays))
        directions = [r[1] for r in results]
        endpoints = [r[2] for r in results]
       
        mean_acc = accs_rays[best_idx]
        std_acc = float(np.std(np.array([r[0] for r in results])[best_idx], ddof=1)) if seeds is not None else 0.0

        

        result = {
            "resolution": resolution,  
            "fmax": fmax,
            "mean_acc": mean_acc,
            "std_acc": std_acc,
            "best_direction": directions[best_idx],
            "best_endpoint": endpoints[best_idx],
        }
        all_results.append(result)

        print(f"[res={resolution:>3}, fmax={fmax}], best_direction={directions[best_idx]}, best_endpoint={endpoints[best_idx]} | "
              f"mean={mean_acc:.2f}% (std={std_acc:.2f}%)")

    else:
        continue


print("\n=== Summary (sorted by mean accuracy) ===")
all_results_sorted = sorted(all_results, key=lambda r: r["mean_acc"], reverse=True)
for r in all_results_sorted:
    print(f"res={r['resolution']:>3}, fmax={r['fmax']}, mean={r['mean_acc']:.2f}% (std={r['std_acc']:.2f}%) \n"
         f"best_direction={r['best_direction']}, best_endpoint={r['best_endpoint']}")

[res= 10, fmax=10], best_direction=[1.4, 1.6, 1.4, 1.6, 1.7, 1.0, 1.8, 2.0, 1.5, 1.1], best_endpoint=[4.5, 3.1, 1.9, 3.6, 4.7, 2.8, 0.1, 0.7, 3.9, 0.2] | mean=86.34% (std=0.00%)
[res= 10, fmax=20], best_direction=[1.9, 1.9, 1.8, 1.3, 1.1, 1.9, 1.6, 1.0, 1.9, 2.0, 1.3, 1.8, 1.1, 1.4, 1.8, 1.4, 1.5, 1.1, 1.8, 1.5], best_endpoint=[1.2, 3.9, 4.9, 2.7, 3.7, 5.0, 0.2, 3.0, 4.8, 0.6, 1.1, 2.8, 3.6, 2.8, 2.7, 0.3, 3.7, 1.6, 0.4, 4.8] | mean=83.41% (std=0.00%)
[res= 10, fmax=40], best_direction=[1.7, 1.8, 1.3, 1.0, 1.6, 1.1, 1.7, 1.3, 1.5, 2.0, 1.8, 1.8, 1.6, 1.9, 1.0, 1.9, 1.4, 1.2, 1.5, 1.9, 1.3, 1.9, 1.5, 2.0, 1.8, 1.8, 1.4, 1.6, 1.2, 1.8, 1.3, 1.7, 1.1, 1.2, 1.4, 1.2, 1.3, 1.2, 1.0, 1.3], best_endpoint=[2.3, 2.1, 4.5, 0.4, 3.1, 1.7, 0.1, 3.9, 2.1, 4.9, 3.9, 4.3, 2.8, 1.7, 4.3, 2.1, 4.4, 4.6, 1.8, 3.6, 4.1, 5.0, 2.3, 1.0, 1.3, 1.9, 0.4, 3.5, 0.3, 2.3, 5.0, 2.0, 1.1, 2.0, 1.3, 1.0, 1.3, 1.5, 2.3, 1.4] | mean=84.39% (std=0.00%)
[res= 50, fmax=10], best_direction=[1.3, 1.0, 1.6, 1.3, 1.3, 1.2, 

In [28]:
%%timeit -n 10

fmax = 10

direction = np.array([1.4, 1.6, 1.4, 1.6, 1.7, 1.0, 1.8, 2.0, 1.5, 1.1])
endpoint  = np.array([4.5, 3.1, 1.9, 3.6, 4.7, 2.8, 0.1, 0.7, 3.9, 0.2])


fft_mag = np.abs(np.fft.fft(data, axis=1))[:, 1:fmax+1]  
base_amp = (2 * np.sqrt(3) * fft_mag) / data.shape[1]   

direction_normal = direction / np.linalg.norm(direction)              
denom = np.sqrt(fmax) * direction_normal                              

birth_global = np.max(-endpoint / denom)                               


deaths = (np.sqrt(3.0) * base_amp[:, :fmax] - endpoint[np.newaxis, :]) / denom[np.newaxis, :]  



mask = (deaths - birth_global) >= 1e-6                             


PDs = []
birth_col = np.full((fmax,), birth_global)           
for i in range(fmax):
    if mask[i].any():
        d = deaths[i, mask[i]].astype(np.float64, copy=False)
        b = birth_col[mask[i]]
        PDs.append(np.asarray(np.stack([b, d], axis=1)))
    else:
        PDs.append(np.asarray([]).reshape(0, 2))

436 μs ± 20.4 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# EMPH-PI (SR)

In [32]:
trials = 100
n_jobs = -1                       
method = 'image'                  



all_results = []  

for resolution in resolutions:
    for fmax in max_frequencies:

 
        fft_mag = np.abs(np.fft.fft(data, axis=1))[:, 1:fmax+1]
        base_amp = (2 * np.sqrt(3) * fft_mag) / data.shape[1]

        for bw in bandwidths:

            results = Parallel(n_jobs=n_jobs, backend="loky", verbose=0)(
                delayed(EMPH_method_original)(
                    tr, resolution, fmax, base_amp, label=label, Ntrain=Ntrain,
                    method=method,
                    bandwidth=bw,      
                    seeds=seeds,
                    max_iter=2000
                )
                for tr in range(trials)
            )

   
            acc_arrays = np.array([r[0] for r in results], dtype=object)  
            accs_rays = np.array([np.mean(a) for a in acc_arrays], dtype=float)
            best_idx = int(np.argmax(accs_rays))

            directions = [r[1] for r in results]
            endpoints  = [r[2] for r in results]

            mean_acc = float(accs_rays[best_idx])
            std_acc = float(np.std(acc_arrays[best_idx], ddof=1)) if seeds is not None else 0.0

            result = {
                "resolution": resolution,
                "fmax": fmax,
                "bandwidth": bw,                     
                "mean_acc": mean_acc,
                "std_acc": std_acc,
                "best_direction": directions[best_idx],
                "best_endpoint": endpoints[best_idx],
            }
            all_results.append(result)

            print(
                f"[res={resolution:>3}, fmax={fmax}, bw={bw}] "
                f"best_direction={directions[best_idx]}, best_endpoint={endpoints[best_idx]} | "
                f"mean={mean_acc:.2f}% (std={std_acc:.2f}%)"
            )


print("\n=== Summary (sorted by mean accuracy) ===")
all_results_sorted = sorted(all_results, key=lambda r: r["mean_acc"], reverse=True)
for r in all_results_sorted:
    print(
        f"res={r['resolution']:>3}, fmax={r['fmax']}, bw={r['bandwidth']} | "
        f"mean={r['mean_acc']:.2f}% (std={r['std_acc']:.2f}%) \n"
        f"best_direction={r['best_direction']}, best_endpoint={r['best_endpoint']}"
    )

[res= 10, fmax=10, bw=0.1] best_direction=[1.9, 1.3, 1.3, 1.2, 1.8, 1.2, 1.8, 1.7, 1.4, 1.5], best_endpoint=[4.5, 4.0, 0.8, 3.7, 2.0, 5.0, 1.1, 2.1, 4.7, 2.9] | mean=86.34% (std=0.00%)
[res= 10, fmax=10, bw=1] best_direction=[1.3, 1.4, 1.6, 1.5, 1.6, 1.6, 1.9, 1.1, 1.7, 1.8], best_endpoint=[3.6, 2.0, 4.7, 0.2, 4.0, 3.0, 0.2, 1.7, 1.9, 0.9] | mean=86.83% (std=0.00%)
[res= 10, fmax=20, bw=0.1] best_direction=[1.6, 1.8, 1.1, 1.4, 1.2, 1.2, 1.0, 2.0, 1.1, 1.9, 1.4, 1.7, 1.1, 1.5, 1.8, 1.1, 1.2, 1.1, 1.3, 1.4], best_endpoint=[1.0, 0.6, 4.5, 3.9, 3.3, 1.2, 0.3, 3.2, 4.7, 4.8, 3.9, 5.0, 0.1, 1.5, 4.9, 0.8, 2.6, 4.6, 3.2, 2.0] | mean=85.37% (std=0.00%)
[res= 10, fmax=20, bw=1] best_direction=[1.5, 1.2, 1.7, 1.6, 1.9, 1.0, 1.8, 1.5, 1.4, 1.5, 1.1, 1.1, 1.7, 1.7, 1.4, 1.5, 1.4, 1.4, 1.2, 1.8], best_endpoint=[2.6, 3.1, 3.2, 1.0, 0.7, 2.9, 2.7, 3.5, 1.7, 1.4, 3.9, 1.4, 0.6, 1.5, 0.5, 2.7, 4.6, 4.1, 2.3, 4.8] | mean=85.85% (std=0.00%)
[res= 10, fmax=40, bw=0.1] best_direction=[1.7, 1.8, 1.3, 1.0, 1

In [31]:
%%timeit -n 10

fmax = 10

direction = np.array([1.9, 1.3, 1.3, 1.2, 1.8, 1.2, 1.8, 1.7, 1.4, 1.5])
endpoint  = np.array([4.5, 4.0, 0.8, 3.7, 2.0, 5.0, 1.1, 2.1, 4.7, 2.9])


fft_mag = np.abs(np.fft.fft(data, axis=1))[:, 1:fmax+1]  
base_amp = (2 * np.sqrt(3) * fft_mag) / data.shape[1]   

direction_normal = direction / np.linalg.norm(direction)              
denom = np.sqrt(fmax) * direction_normal                              

birth_global = np.max(-endpoint / denom)                               


deaths = (np.sqrt(3.0) * base_amp[:, :fmax] - endpoint[np.newaxis, :]) / denom[np.newaxis, :]  



mask = (deaths - birth_global) >= 1e-6                             


PDs = []
birth_col = np.full((fmax,), birth_global)           
for i in range(fmax):
    if mask[i].any():
        d = deaths[i, mask[i]].astype(np.float64, copy=False)
        b = birth_col[mask[i]]
        PDs.append(np.asarray(np.stack([b, d], axis=1)))
    else:
        PDs.append(np.asarray([]).reshape(0, 2))

461 μs ± 39.7 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# EMPH-Betti (CR)

In [33]:
trials = 100
n_jobs = -1                     
method = 'betti'                



all_results = []  

for resolution in resolutions:
    for fmax in max_frequencies:
        

        fft_mag = np.abs(np.fft.fft(data, axis=1))[:, 1:fmax+1]  
        base_amp = (2 * np.sqrt(3) * fft_mag) / data.shape[1]               

        results = Parallel(n_jobs=n_jobs, backend="loky", verbose=0)(
            delayed(EMPH_method)(
                tr, resolution, fmax, base_amp, label=label, Ntrain=Ntrain,
                method=method,
               seeds = seeds
            )
            for tr in range(trials)
        )


        accs_rays = np.mean(np.array([r[0] for r in results]), axis=1)
        best_idx = int(np.argmax(accs_rays))
        directions = [r[1] for r in results]
        endpoints = [r[2] for r in results]
       
        mean_acc = accs_rays[best_idx]
        std_acc = float(np.std(np.array([r[0] for r in results])[best_idx], ddof=1)) if seeds is not None else 0.0

        

        result = {
            "resolution": resolution,  
            "fmax": fmax,
            "mean_acc": mean_acc,
            "std_acc": std_acc,
            "best_direction": directions[best_idx],
            "best_endpoint": endpoints[best_idx],
        }
        all_results.append(result)

        print(f"[res={resolution:>3}, fmax={fmax}], best_direction={directions[best_idx]}, best_endpoint={endpoints[best_idx]} | "
              f"mean={mean_acc:.2f}% (std={std_acc:.2f}%)")

    else:
        continue


print("\n=== Summary (sorted by mean accuracy) ===")
all_results_sorted = sorted(all_results, key=lambda r: r["mean_acc"], reverse=True)
for r in all_results_sorted:
    print(f"res={r['resolution']:>3}, fmax={r['fmax']}, mean={r['mean_acc']:.2f}% (std={r['std_acc']:.2f}%) \n"
         f"best_direction={r['best_direction']}, best_endpoint={r['best_endpoint']}")

[res= 10, fmax=10], best_direction=[1.2, 1.5, 1.1, 1.6, 1.1, 1.5, 1.7, 1.1, 1.9, 1.9], best_endpoint=[2.4, 3.6, 0.7, 0.9, 3.8, 1.9, 4.1, 4.7, 1.1, 3.3] | mean=86.34% (std=0.00%)
[res= 10, fmax=20], best_direction=[1.9, 1.3, 1.6, 1.8, 1.7, 1.9, 1.9, 1.9, 1.0, 1.4, 1.5, 1.1, 1.0, 1.8, 2.0, 1.8, 1.3, 1.7, 1.3, 1.7], best_endpoint=[1.4, 3.9, 4.9, 4.9, 4.4, 4.6, 3.5, 2.8, 4.6, 0.4, 1.9, 3.0, 2.4, 4.0, 1.0, 0.3, 0.6, 4.8, 0.7, 1.1] | mean=85.85% (std=0.00%)
[res= 10, fmax=40], best_direction=[1.7, 1.7, 1.1, 1.7, 1.3, 1.5, 2.0, 1.1, 1.2, 1.1, 1.8, 1.7, 1.2, 1.1, 1.7, 1.2, 1.3, 1.7, 1.1, 1.3, 1.5, 1.6, 1.4, 2.0, 1.8, 1.0, 1.3, 1.0, 1.8, 1.1, 1.4, 1.8, 1.4, 1.0, 1.3, 1.3, 1.6, 1.8, 1.5, 1.0], best_endpoint=[4.2, 2.1, 3.8, 1.2, 0.2, 1.8, 4.0, 4.8, 2.8, 3.9, 3.7, 0.2, 1.3, 0.6, 3.6, 0.6, 1.8, 1.0, 4.5, 4.0, 3.0, 1.1, 3.2, 2.4, 2.7, 4.4, 3.1, 1.4, 0.2, 1.7, 0.9, 1.1, 1.1, 1.3, 2.2, 1.4, 4.1, 2.8, 1.4, 3.0] | mean=85.85% (std=0.00%)
[res= 50, fmax=10], best_direction=[1.5, 2.0, 1.1, 1.9, 1.3, 1.4, 

In [34]:
%%timeit -n 10

fmax = 10

direction = np.array([1.2, 1.5, 1.1, 1.6, 1.1, 1.5, 1.7, 1.1, 1.9, 1.9])
endpoint  = np.array([2.4, 3.6, 0.7, 0.9, 3.8, 1.9, 4.1, 4.7, 1.1, 3.3])


dir_norm = np.sqrt(np.dot(direction, direction))
inv_sqrt_fmax = 1.0 / np.sqrt(fmax)
scale = (dir_norm * inv_sqrt_fmax) / direction                                     


fft_mag = np.abs(np.fft.fft(data, axis=1))[:, 1:fmax+1]  
base_amp = (2 * np.sqrt(3) * fft_mag) / data.shape[1]  

birth_time = -endpoint * scale                     
death_time = ((base_amp - endpoint) * scale)


PDs = np.empty((base_amp.shape[0], fmax, 2))
PDs[..., 0] = birth_time     # birth
PDs[..., 1] = death_time     # death

365 μs ± 28.3 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# EMPH-PI (CR)

In [35]:
trials = 100
n_jobs = -1                        
method = 'image'                 



all_results = [] 

for resolution in resolutions:
    for fmax in max_frequencies:

  
        fft_mag = np.abs(np.fft.fft(data, axis=1))[:, 1:fmax+1]
        base_amp = (2 * np.sqrt(3) * fft_mag) / data.shape[1]

        for bw in bandwidths:
           
            results = Parallel(n_jobs=n_jobs, backend="loky", verbose=0)(
                delayed(EMPH_method)(
                    tr, resolution, fmax, base_amp, label=label, Ntrain=Ntrain,
                    method=method,
                    bandwidth=bw,      # <-- 추가
                    seeds=seeds
                )
                for tr in range(trials)
            )

   
            acc_arrays = np.array([r[0] for r in results], dtype=object) 
            accs_rays = np.array([np.mean(a) for a in acc_arrays], dtype=float)
            best_idx = int(np.argmax(accs_rays))

            directions = [r[1] for r in results]
            endpoints  = [r[2] for r in results]

            mean_acc = float(accs_rays[best_idx])
            std_acc = float(np.std(acc_arrays[best_idx], ddof=1)) if seeds is not None else 0.0

            result = {
                "resolution": resolution,
                "fmax": fmax,
                "bandwidth": bw,                   
                "mean_acc": mean_acc,
                "std_acc": std_acc,
                "best_direction": directions[best_idx],
                "best_endpoint": endpoints[best_idx],
            }
            all_results.append(result)

            print(
                f"[res={resolution:>3}, fmax={fmax}, bw={bw}] "
                f"best_direction={directions[best_idx]}, best_endpoint={endpoints[best_idx]} | "
                f"mean={mean_acc:.2f}% (std={std_acc:.2f}%)"
            )


print("\n=== Summary (sorted by mean accuracy) ===")
all_results_sorted = sorted(all_results, key=lambda r: r["mean_acc"], reverse=True)
for r in all_results_sorted:
    print(
        f"res={r['resolution']:>3}, fmax={r['fmax']}, bw={r['bandwidth']} | "
        f"mean={r['mean_acc']:.2f}% (std={r['std_acc']:.2f}%) \n"
        f"best_direction={r['best_direction']}, best_endpoint={r['best_endpoint']}"
    )

[res= 10, fmax=10, bw=0.1] best_direction=[1.3, 1.9, 1.2, 1.2, 1.3, 1.2, 1.7, 1.1, 1.9, 1.9], best_endpoint=[0.0, 2.7, 0.5, 1.3, 2.1, 2.3, 2.3, 4.6, 1.3, 0.9] | mean=87.32% (std=0.00%)
[res= 10, fmax=10, bw=1] best_direction=[1.2, 1.7, 1.7, 1.7, 1.7, 1.1, 1.2, 1.3, 1.2, 1.1], best_endpoint=[1.8, 3.4, 3.1, 1.1, 4.3, 1.4, 4.6, 2.8, 4.8, 1.6] | mean=87.80% (std=0.00%)
[res= 10, fmax=20, bw=0.1] best_direction=[1.3, 2.0, 1.3, 1.8, 1.9, 1.4, 1.4, 1.4, 1.1, 1.5, 1.2, 1.3, 1.2, 1.2, 1.8, 1.4, 1.3, 1.6, 1.6, 1.6], best_endpoint=[4.6, 0.8, 1.9, 1.4, 0.1, 0.9, 2.0, 2.0, 3.1, 2.3, 3.0, 1.1, 0.7, 1.6, 0.5, 1.8, 2.3, 3.6, 4.3, 0.3] | mean=86.34% (std=0.00%)
[res= 10, fmax=20, bw=1] best_direction=[1.4, 1.2, 1.9, 1.6, 1.4, 1.5, 1.5, 1.1, 1.3, 1.0, 1.9, 1.7, 1.4, 2.0, 1.3, 1.7, 1.8, 1.2, 1.5, 1.2], best_endpoint=[2.0, 2.1, 1.0, 4.4, 4.5, 0.2, 4.9, 0.2, 4.6, 1.1, 3.8, 0.6, 4.6, 1.4, 1.4, 1.6, 4.8, 0.8, 0.6, 1.5] | mean=86.34% (std=0.00%)
[res= 10, fmax=40, bw=0.1] best_direction=[1.9, 1.3, 1.6, 1.8, 1

In [36]:
%%timeit -n 10

fmax = 10

direction = np.array([1.2, 1.7, 1.7, 1.7, 1.7, 1.1, 1.2, 1.3, 1.2, 1.1])
endpoint  = np.array([1.8, 3.4, 3.1, 1.1, 4.3, 1.4, 4.6, 2.8, 4.8, 1.6])


dir_norm = np.sqrt(np.dot(direction, direction))
inv_sqrt_fmax = 1.0 / np.sqrt(fmax)
scale = (dir_norm * inv_sqrt_fmax) / direction                                     



fft_mag = np.abs(np.fft.fft(data, axis=1))[:, 1:fmax+1]  
base_amp = (2 * np.sqrt(3) * fft_mag) / data.shape[1]  

birth_time = -endpoint * scale                     
death_time = ((base_amp - endpoint) * scale)


PDs = np.empty((base_amp.shape[0], fmax, 2))
PDs[..., 0] = birth_time    
PDs[..., 1] = death_time    

346 μs ± 35.8 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
